In [3]:
import geopandas
import pandas as pd

In [4]:
df_osm = pd.read_parquet("../data/transformed/poi_clean_category.parquet")

In [5]:
gdf_osm = geopandas.GeoDataFrame(
    df_osm, geometry=geopandas.points_from_xy(df_osm.lon, df_osm.lat), crs="EPSG:4326"
)

In [6]:
gdf_osm.to_parquet("../data/transformed/poi_clean_category_geo.parquet")

In [7]:
gdf_ot = geopandas.read_parquet("../data/raw/OFFRES_TOURISTIQUES.parquet")

In [8]:
gdf_osm.head(5)

,type,id,lat,lon,addr:city,addr:country,addr:housenumber,addr:postcode,addr:street,brand,...,payment:cash,payment:maestro,payment:mastercard,contact:mobile,url,contact:email,addr:suburb,sub_category,category,geometry
0,node,23656136,49.858471,6.364970,None,None,None,None,None,None,...,None,None,None,None,None,None,None,fitness_station,leisure,POINT (6.36497 49.85847)
1,node,26860223,49.743478,6.089852,None,None,None,None,None,None,...,None,None,None,None,None,None,None,restaurant,amenity,POINT (6.08985 49.74348)
2,node,30432808,50.856197,5.825697,None,None,None,None,None,None,...,None,None,None,None,None,None,None,attraction,tourism,POINT (5.82570 50.85620)
3,node,31425173,50.844029,5.689079,None,None,None,None,None,None,...,None,None,None,None,None,None,None,city_gate,historic,POINT (5.68908 50.84403)
4,node,31575884,50.844551,5.690282,None,None,None,None,None,None,...,None,None,None,None,None,None,None,library,amenity,POINT (5.69028 50.84455)


In [9]:
gdf_ot.head(5)

,OBJECTID,ID_OFFRE,TYPE_OFFRE,CODE,NOM,RUE,NUMERO,CODE_POSTAL,COMMUNE,PHONE1,MAIL,SITE_WEB,FACEBOOK,X,Y,LON,LAT,ORGANISME,DATE_MODIFICATION,geometry
0,11479.0,11.0,Découverte et Divertissement,LOD-A1-002M-282Y,Centre culturel,Rue Saint-Pierre,6,4690,Bassenge,None,None,None,None,232964.0,160811.0,5.54446,50.75189,Pays de Liège,2022-11-15 00:00:00+00:00,POINT (232964.000 160811.001)
1,11460.0,11.0,Découverte et Divertissement,LOD-A1-002M-26W4,Église Saint-Victor,Rue de la Dérivation,2,4690,Bassenge,None,None,None,None,233086.0,160743.0,5.54617,50.75127,Pays de Liège,2022-11-15 00:00:00+00:00,POINT (233086.000 160743.001)
2,11464.0,11.0,Découverte et Divertissement,LOD-A1-002M-271F,Maison Van der Wielen,Rue Henri Vander Wielen,1,4690,Bassenge,None,None,None,None,233478.0,160423.0,5.55166,50.74833,Pays de Liège,2022-11-15 00:00:00+00:00,POINT (233478.000 160423.001)
3,11471.0,11.0,Découverte et Divertissement,LOD-A1-002M-27BD,Cimetière de Glons,Rue de Brus,10,4690,Bassenge,None,None,None,None,233745.0,160258.0,5.55541,50.74681,Pays de Liège,2022-11-15 00:00:00+00:00,POINT (233745.000 160258.001)
4,15243.0,261.0,Restauration,RST-17-001V-0003,Le Croq midi,Rue du Commerce,2,4690,Bassenge,+3242787047,None,None,None,236401.0,161377.0,5.59329,50.75648,Pays de Liège,2019-05-15 00:00:00+00:00,POINT (236401.000 161377.001)


In [10]:
# perform spatial join
joined = gdf_osm.to_crs("EPSG:3857").sjoin_nearest(
    gdf_ot.to_crs("EPSG:3857"), how="inner", lsuffix="osm", rsuffix="ot", max_distance=2
)
print(joined.shape)

(146, 104)


In [14]:
link = gdf_osm[gdf_osm["image"].notnull()]
link.shape

(3278, 84)

In [15]:
import folium

# create a map centered on the first point of gdf_osm
m = folium.Map(location=[link.iloc[0].lat, link.iloc[0].lon], zoom_start=12)

# add markers for each point in joined dataframe
for _, row in link.iterrows():
    folium.Marker(location=[row["lat"], row["lon"]], popup=row["id"]).add_to(m)

# display the map
m

In [ ]:
import folium

# create a map centered on the first point of gdf_osm
m = folium.Map(location=[gdf_osm.iloc[0].lat, gdf_osm.iloc[0].lon], zoom_start=12)

# add markers for each point in joined dataframe
for _, row in joined.iterrows():
    popup = row["name"] + " - " + row["NOM"]
    folium.Marker(location=[row["LAT"], row["LON"]], popup=row["NOM"]).add_to(m)
    folium.Marker(location=[row["lat"], row["lon"]], popup=row["name"]).add_to(m)

# display the map
m